In [12]:
import hashlib
import hmac
import json
import os, requests

from xAPIConnector import BitkubClient
from dotenv import load_dotenv
load_dotenv()

True

In [45]:
API_HOST = 'https://api.bitkub.com'
API_KEY = os.getenv('BITKUB_API_KEY')
API_SECRET = bytes(os.getenv('BITKUB_SECRET'), 'utf-8')
# API_KEY, API_SECRET

In [46]:
def json_encode(data):
    return json.dumps(data, separators=(',', ':'), sort_keys=True)

In [47]:
def sign(data):
    j = json_encode(data)
    print('Signing payload: ' + j)
    h = hmac.new(API_SECRET, msg=j.encode(), digestmod=hashlib.sha256)
    return h.hexdigest()

In [55]:
# check server time
response = requests.get(API_HOST + '/api/servertime')
ts = int(response.text)
print('Server time: ' + response.text)

Server time: 1649410190


In [56]:
# place ask
header = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'X-BTK-APIKEY': API_KEY,
}
data = {
    'sym': 'THB_BTC',
    'amt': 0.0001, # BTC amount you want to sell
    'rat': 1475998, # price
    'typ': 'limit', # limit or market (in case market set rat(price) to 0)
    'ts': ts,
}
signature = sign(data)
data['sig'] = signature

print('Payload with signature: ' + json_encode(data))
response = requests.post(API_HOST + '/api/market/place-ask/test', headers=header, data=json_encode(data))

print('Response: ' + response.text)

Signing payload: {"amt":0.0001,"rat":1475998,"sym":"THB_BTC","ts":1649410190,"typ":"limit"}
Payload with signature: {"amt":0.0001,"rat":1475998,"sig":"3d6e661886c1040a2f51a1e79d853210eb56543904d7c78c73a4489b06619e2a","sym":"THB_BTC","ts":1649410190,"typ":"limit"}
Response: {"error":0,"result":{"id":0,"hash":"test","typ":"limit","amt":0.0001,"rat":1475998,"fee":0.37,"cre":0,"rec":147.22,"ts":1649410191}}


In [15]:
dict(a=0, b=1)

{'a': 0, 'b': 1}

In [24]:
def getMarketTicker(symbol=None):
    url = API_HOST + '/api/market/ticker'
    if symbol != None:
        response = requests.get(url, params={'sym': symbol})
    else:
        response = requests.get(url)
    return json.loads(response.text)

In [25]:
res = getMarketTicker('THB_BTC')
res

{'THB_BTC': {'id': 1,
  'last': 1480919.94,
  'lowestAsk': 1480875,
  'highestBid': 1480000,
  'percentChange': 0.88,
  'baseVolume': 212.16731697,
  'quoteVolume': 311451603.93,
  'isFrozen': 0,
  'high24hr': 1483880,
  'low24hr': 1454001.01,
  'change': 12920.09,
  'prevClose': 1480919.94,
  'prevOpen': 1467999.85}}

In [27]:
res['THB_BTC']['lowestAsk']

1480875